In [1]:
import numpy as np
import pandas as pd

In [40]:
path = "/Users/dj/Documents/GitHub/Causal_Data_Science/Project/Data/lucas0_text/"

In [41]:
# reading csv files
train =  pd.read_csv(path + 'lucas0_train.data', sep=" ", header = None)
test =  pd.read_csv(path + 'lucas0_test.data', sep=" ", header = None)
target =  pd.read_csv(path + 'lucas0_train.targets', sep=" ", header = None)
# param =  pd.read_csv(path + 'lucas0.param', sep=" ")

In [42]:
print(train.sum().sum())
print(test.sum().sum())

11923.0
60290.0


In [43]:
print(train.shape)
print(test.shape)
print(target.shape)

(2000, 12)
(10000, 12)
(2000, 1)
